In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = (SparkSession.builder
         .appName("perform-aagregations")
         .master("spark://spark-master:7077")
         .config("spark.executor.memory", "512m")
         .getOrCreate())

spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/20 11:18:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = (spark.read.format("csv")
      .option("header", "true")
      .option("nullValue", "null")
      .option("dateFormat", "LLLL d, y")
      .load("../data/netflix_titles.csv"))

In [3]:
# Group the data by a column
grouped_df = df.groupBy("country")

In [4]:
# Count the number of rows in each group
count_df = grouped_df.count()
count_df.show()

+--------------------+-----+
|             country|count|
+--------------------+-----+
|Peru, United Stat...|    1|
|India, United Kin...|    1|
|Japan, Canada, Un...|    1|
|United Kingdom, C...|    1|
|      India, Germany|    2|
|South Africa, Uni...|    1|
|              Russia|   15|
|United Kingdom, G...|    1|
|Chile, United Sta...|    1|
|South Africa, Angola|    1|
|United States, Po...|    1|
|  Philippines, Qatar|    1|
|Hong Kong, China,...|    1|
|  Germany, Sri Lanka|    1|
|Denmark, France, ...|    1|
|United States, Fr...|    1|
|United Kingdom, N...|    2|
|Australia, United...|    2|
|Brazil, France, G...|    1|
|United States, Ir...|    3|
+--------------------+-----+
only showing top 20 rows



In [5]:
# Apply custom aggregation using max
max_release_df = grouped_df.agg(max(col("date_added")))
max_release_df.show()

+--------------------+--------------------+
|             country|     max(date_added)|
+--------------------+--------------------+
|                null|   September 9, 2021|
|     Ama K. Abebrese|  Kobina Amissah Sam|
|         Aziz Ansari|         Carla Gallo|
|            Chuck D.|     Desiree Densiti|
|       Dominic Costa|        Nick Ferraro|
|          Doug Plaut|    Cheyenne Jackson|
|     Francesc Orella|        Imanol Arias|
|  Henri-Noël Tabary"|              France|
|       James Toback"|       United States|
| Justin ""Alyssa ...|         Molly Ryman|
|  Lachion Buckingham|       Chasity Moore|
|  Leonardo Sbaraglia|     Francesc Orella|
|   Michael Cavalieri|     Walton Goggins"|
| Remilekun ""Remi...| Charles  ""Charl...|
|          Rob Morgan|     Shakira Barrera|
|       Sophia Loren"|       United States|
|     Tantoo Cardinal|   Robert Pastorelli|
|      Theo Campbell"|       United States|
| Tobechukwu ""iLL...|          Toni Tones|
| plus Whitney Cum...|          

In [6]:
release_date_gouped_df = (
    df.groupBy("country")
    .agg(
        count("show_id").alias("NumberOfReleases")
        ,max("date_added").alias("LastReleaseDate")
        ,min("date_added").alias("FirstReleaseDate")))

release_date_gouped_df.show()

+--------------------+----------------+--------------------+--------------------+
|             country|NumberOfReleases|     LastReleaseDate|    FirstReleaseDate|
+--------------------+----------------+--------------------+--------------------+
|                null|             832|   September 9, 2021|   December 14, 2018|
|     Ama K. Abebrese|               1|  Kobina Amissah Sam|  Kobina Amissah Sam|
|         Aziz Ansari|               1|         Carla Gallo|         Carla Gallo|
|            Chuck D.|               1|     Desiree Densiti|     Desiree Densiti|
|       Dominic Costa|               1|        Nick Ferraro|        Nick Ferraro|
|          Doug Plaut|               1|    Cheyenne Jackson|    Cheyenne Jackson|
|     Francesc Orella|               1|        Imanol Arias|        Imanol Arias|
|  Henri-Noël Tabary"|               1|              France|              France|
|       James Toback"|               1|       United States|       United States|
| Justin ""Alyss

### Pivot Tables

In [7]:
pivot_table = (df.groupBy("country").pivot("type").agg(count("show_id")))
pivot_table.show()

+--------------------+----+-----+-------+-------------+
|             country|null|Movie|TV Show|William Wyler|
+--------------------+----+-----+-------+-------------+
|Peru, United Stat...|null|    1|   null|         null|
|United Kingdom, C...|null| null|      1|         null|
|India, United Kin...|null|    1|   null|         null|
|      India, Germany|null|    2|   null|         null|
|Japan, Canada, Un...|null| null|      1|         null|
|South Africa, Uni...|null|    1|   null|         null|
|              Russia|null|    1|     14|         null|
|United Kingdom, G...|null| null|      1|         null|
|Chile, United Sta...|null|    1|   null|         null|
|  Philippines, Qatar|null|    1|   null|         null|
|United States, Fr...|null| null|      1|         null|
|Hong Kong, China,...|null|    1|   null|         null|
|Denmark, France, ...|null|    1|   null|         null|
|South Africa, Angola|null|    1|   null|         null|
|United States, Po...|null| null|      1|       

### Approximate Aggregations

In [14]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

# Define a Schema
schema = StructType([
    StructField("Id", IntegerType(), True),
    StructField("ProductId", StringType(), True),
    StructField("UserId", StringType(), True),
    StructField("ProfileName", StringType(), True),
    StructField("HelpfulnessNumerator", StringType(), True),
    StructField("HelpfulnessDenominator", StringType(), True),
    StructField("Score", IntegerType(), True),
    StructField("Time", StringType(), True),
    StructField("Summary", StringType(), True),
    StructField("Text", StringType(), True)])


review_df = (spark.read.format("csv")
      .option("header",True)
      .schema(schema)
      .load("../data/Reviews.csv"))

In [15]:
# Approximate quantile calculation
quantiles = review_df.approxQuantile("Score", [0.25, 0.5, 0.75], 0.1)
print("Approximate Quantiles:", quantiles)

Approximate Quantiles: [4.0, 5.0, 5.0]


In [16]:
# Approximate distinct count calculation
approx_distinct_count = review_df.select(approxCountDistinct("ProductId", rsd=0.1).alias("approx_distinct_count"))
approx_distinct_count.show()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/functions.py:1075: FutureWarning: Deprecated in 2.1, use approx_count_distinct instead.
  warnings.warn("Deprecated in 2.1, use approx_count_distinct instead.", FutureWarning)


+---------------------+
|approx_distinct_count|
+---------------------+
|                14801|
+---------------------+



In [17]:
spark.stop()